# Método de Benjamini–Hochberg

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%load_ext rpy2.ipython

In [0]:
%%R
install.packages('data.table')
library(data.table)

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: data.table 1.12.2 using 1 threads (see ?getDTthreads).  Latest news: r-datatable.com

  warnings.warn(x, RRuntimeWarning)


In [0]:
%%R
W = as.matrix(read.table("wadj_py.csv", sep = ","))
P = as.matrix(read.table("pVal_py.csv", sep = ","))

In [0]:
%%R
len <- 1:(ncol(W)*nrow(W))
n_rows = nrow(W)
n_cols = ncol(W)

In [0]:
%%R
df <- data.frame(index = len, corr = as.vector(W), pval = as.vector(P))
rm(W)
rm(P)
head(df)

  index      corr         pval
1     1 1.0000000 0.000000e+00
2     2 0.8204334 3.042081e-05
3     3 0.6573787 3.030290e-03
4     4 0.4179567 8.435265e-02
5     5 0.4427245 6.578236e-02
6     6 0.5294118 2.385823e-02


In [0]:
%%R
setorder(df, pval) 
#df <- df %>% arrange(pval)
head(df)

      index corr pval
1         1    1    0
8402   8402    1    0
16803 16803    1    0
25204 25204    1    0
33605 33605    1    0
42006 42006    1    0


In [0]:
%%R
df$rank <- len
head(df)

      index corr pval rank
1         1    1    0    1
8402   8402    1    0    2
16803 16803    1    0    3
25204 25204    1    0    4
33605 33605    1    0    5
42006 42006    1    0    6


In [0]:
%%R
n_tests <- length(len)
Q <- .10
df$critical <- df$rank*Q/n_tests
head(df)

      index corr pval rank     critical
1         1    1    0    1 1.417234e-09
8402   8402    1    0    2 2.834467e-09
16803 16803    1    0    3 4.251701e-09
25204 25204    1    0    4 5.668934e-09
33605 33605    1    0    5 7.086168e-09
42006 42006    1    0    6 8.503401e-09


In [0]:
%%R
pval <- df$pval
crit <- df$critical

df$critical <- NULL
df$pval <- NULL

head(df)

      index corr rank
1         1    1    1
8402   8402    1    2
16803 16803    1    3
25204 25204    1    4
33605 33605    1    5
42006 42006    1    6


In [0]:
%%R
for(i in length(len):1){
    if(pval[i] < crit[i]){
        buff <- i
        break
    }
}
print(buff)

[1] 20361806


In [0]:
%%R
df$sig <- 0
df$sig[df$rank <= buff] <- 1
df$rank <- NULL

In [0]:
%%R
head(df)

      index corr sig
1         1    1   1
8402   8402    1   1
16803 16803    1   1
25204 25204    1   1
33605 33605    1   1
42006 42006    1   1


In [0]:
%%R
rm(pval)
rm(crit)

In [0]:
%%R
df[(buff-2):(buff+3),]

            index      corr sig
15365447 15365447 0.5146754   1
31196618 31196618 0.5146754   1
39601323 39601323 0.5146754   1
51333042 51333042 0.5146626   0
51338551 51338551 0.5146626   0
5390512   5390512 0.5146626   0


In [0]:
%%R
df$corr[df$sig == 0] <- 0

In [0]:
%%R
setorder(df, index)  
#df <- df %>% arrange(index)
head(df)

  index      corr sig
1     1 1.0000000   1
2     2 0.8204334   1
3     3 0.6573787   1
4     4 0.0000000   0
5     5 0.0000000   0
6     6 0.5294118   1


In [0]:
%%R
write.csv(df, "data_frame.csv")

In [0]:
%%R
df$index <- NULL

In [0]:
%%R
corr <- df$corr
sig <- df$sig
rm(df)

In [0]:
%%R
W <- matrix(corr, n_rows, n_cols)
A <- matrix(sig, n_rows, n_cols)

In [0]:
%%R
W[1:5,1:5]

          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 1.0000000 0.8204334 0.6573787 0.0000000 0.0000000
[2,] 0.8204334 1.0000000 0.0000000 0.0000000 0.5995872
[3,] 0.6573787 0.0000000 1.0000000 0.0000000 0.0000000
[4,] 0.0000000 0.0000000 0.0000000 1.0000000 0.5459236
[5,] 0.0000000 0.5995872 0.0000000 0.5459236 1.0000000


In [0]:
%%R
for(i in 1:n_cols){
    W[i,i] <- 0
    A[i,i] <- 0
}

In [0]:
%%R
W[1:5,1:5]

          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 0.0000000 0.8204334 0.6573787 0.0000000 0.0000000
[2,] 0.8204334 0.0000000 0.0000000 0.0000000 0.5995872
[3,] 0.6573787 0.0000000 0.0000000 0.0000000 0.0000000
[4,] 0.0000000 0.0000000 0.0000000 0.0000000 0.5459236
[5,] 0.0000000 0.5995872 0.0000000 0.5459236 0.0000000


In [0]:
%%R
write.csv(W, "weighted_adjacency_matrix.csv")
write.csv(A, "adjacency_matrix.csv")